# Word Embedding

Word Embedding is a type of word representation that allows words with similar meaning to have a similar representation. It is a learned representation for text where words that have the same meaning have a similar representation. The process of creating word embeddings is to train a model on a large body of text using unsupervised learning so that it learns to predict a word given its context, or vice versa. This results in embeddings where words that are close together in the high-dimensional embedding space are expected to be semantically similar.

Notably, word embeddings do not just assign each word to a unique vector, but they actually represent words in a way that captures semantic or syntactic similarity, based on the text corpus the model is trained on. Embedding vectors are often hundreds of dimensions and are good at capturing nuanced relationships between words.

![word_embed](../asset/word_embed.png)

## Embedding layer

An Embedding Layer in machine learning is a layer that creates word embeddings from input sequences by mapping words or integers to dense vectors of real numbers. It's often used in natural language processing tasks where the layer takes in a 2D input of integer sequences (word indices) and on output provides 3D floating point tensor sequences. Each individual sequence in the 3D output tensor will have the same length as the original input sequence, but each word or integer in the sequence is now represented by a dense vector. These vectors capture semantic relationships between words, and their dimensionality is a hyperparameter that can be tuned for specific tasks.

### Very basic usage

Embedding is just a simple lookup table. Link a index to a trainable vector.




In [ ]:
import torch
import torch.nn as nn

word_to_ix = {"hello": 0, "world": 1}

# 2 words in vocab "hello" and "world"
# 5 dimensional embeddings
embeds = nn.Embedding(num_embeddings=2, embedding_dim=5)

lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)

hello_embed = embeds(lookup_tensor)

print(hello_embed)

Get train parameters

In [ ]:
for param in embeds.parameters():
  print(param)

Just for fun set weight to a fix value

In [ ]:
embedding_lookup = torch.tensor([
    [1,0,0,0,1],
    [0,1,1,1,0],
], dtype=torch.float32)
embeds.weight = nn.Parameter(embedding_lookup)
embeds.weight

As you can see, if I select index 0 or 1, I get my embedding_lookup line.

In [ ]:
print(embeds(torch.tensor([0])))
print(embeds(torch.tensor([1])))

## GPT embedding

Let's get the very first GPT model and see how the embedding layer size

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

print("vocab size", tokenizer.vocab_size)

# expected Embedding(50257, 768)
# 50257 = vocabulary size
# 768 = number of features
print("Embedding size", model.wte)


[Let's look at the code for this model](https://github.com/huggingface/transformers/blob/v4.25.1/src/transformers/models/gpt2/modeling_gpt2.py#L667)

## Train first embedding layer

For this part we will train our first embedding layer on disney park reviews !

To start we will ony train this layer on letter instance of words



In [1]:
import pandas as pd

# Load and take subset of the dataset
df = pd.read_csv("data/disney_review/train.csv")[:5000]
df.head()

,Unnamed: 0,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,16833,333121929,5,2015-12,United States,Truly the most magical place on the West Coast...,Disneyland_California
1,28602,71690002,5,missing,Canada,"I cry every time we have to leave this place, ...",Disneyland_California
2,32094,481683772,5,2017-4,United Kingdom,"Staff everywhere were just friendly, the custo...",Disneyland_Paris
3,16576,339637317,5,2016-1,United States,"It's Disneyland, what more is there to say? W...",Disneyland_California
4,13962,438710000,5,2016-10,United Kingdom,We visited this park at Halloween and enjoyed ...,Disneyland_California


In [ ]:
# Count each deffrent values of rating
df['Rating'].value_counts()

In [ ]:
# Get reviews
reviews = df["Review_Text"].values.tolist()

Now we will creat an sequence of letter based on sentences

For example:

```
aba decides a
```

will produce

```
[
  ('a', 'b'),
  ('b', 'a'),
  ('a', ' '),
  (' ', 'd'),
  ('d', 'e'),
  ('e', 'c'),
  ('c', 'i'),
  ('i', 'd'),
  ('d', 'e'),
  ('e', 's'),
  ('s', ' '),
  (' ', 'a'),
]
```

In [ ]:
import itertools as it
import re

def sliding_window(txt):
  for i in range(len(txt) - 1):
    yield txt[i], txt[i + 1]
    
window = []
for title in reviews:
  # Get only a to z and 0 to 9 letters and numerb
  title = re.sub('[^a-zA-Z0-9]+', '', title.lower())
  window.append(sliding_window(title))
window = list(it.chain(*window))

# Number of window
print(len(window))
# Get first 5 example
window[:5]

Now lets do a one hot encoding so a letter = an id (like sql table id)

```
{' ': 2,
 'a': 0,
 'b': 1,
 'c': 5,
 'd': 3,
 'e': 4,
 'g': 8,
 'i': 6,
 'l': 16,
 'm': 12,
 'n': 9,
 'o': 11,
 'r': 15,
 's': 7,
 't': 10,
 'u': 13,
 'y': 14}
```

In [ ]:
import numpy as np
mapping = {c: i for i, c in enumerate(pd.DataFrame(window)[0].unique())}
integers_in = np.array([mapping[w[0]] for w in window])
integers_out = np.array([mapping[w[1]] for w in window])

print("Shape of input", integers_in.shape)
print("Input example", integers_in[0], integers_out[0])
print("Show generate mapping\n", mapping)

### The dataset class
The dataset class summarizes the basic functionality of a dataset in a natural way. To define a dataset in PyTorch, we simply specify two functions: __getitem__, and __len__. The get-item function has to return the i-th data point in the dataset, while the len function returns the size of the dataset.

In [ ]:
import torch.utils.data as data

class NextLetterDataset(data.Dataset):

  def __init__(self, integers_in, integers_out):
    self.integers_in = integers_in
    self.integers_out = integers_out
  
  def __len__(self):
    # Number of data pin dataset
    return len(self.integers_in)

  def __getitem__(self, idx):
      # Return the idx-th data point of the dataset
      # If we have multiple things to return (data point and label), we can return them as tuple
      data_point = self.integers_in[idx]
      data_label = self.integers_out[idx]
      return torch.tensor(data_point), torch.tensor(data_label)

### Build first embedding model

We will build a simple network to do next letter prediction

![next letter](../asset/next_letter_prediction.png)

In [ ]:
import torch.nn.functional as F

class NextLetterPrediction(torch.nn.Module):
   def __init__(self, vocab_size, embedding_size):
       super(NextLetterPrediction, self).__init__()
       self.embedding = torch.nn.Embedding(vocab_size, embedding_size)
       self.fc = torch.nn.Linear(embedding_size, vocab_size)

   def forward(self, x):
       x = F.relu(self.embedding(x))
       x = self.fc(x)
       return x

### Plot letter before the train

Let's plot letter embedding before to train the model

In [ ]:
model = NextLetterPrediction(vocab_size=len(mapping),
                             # For X and Y plot
                             embedding_size=2)

In [ ]:
import matplotlib.pylab as plt

idx_to_calc = list(mapping.values())
idx_to_calc = np.array([idx_to_calc]).T

translator = {v:k for k,v in mapping.items()}
preds = model.embedding(torch.tensor(idx_to_calc)).detach().numpy()
plt.scatter(preds[:, 0, 0], preds[:, 0, 1], alpha=0)
for i, idx in enumerate(idx_to_calc):
  plt.text(preds[i, 0, 0], preds[i, 0, 1], translator[idx[0]])

### Train loop


In [ ]:
# Init the dataset into the DataLoader
dataset = NextLetterDataset(integers_in, integers_out)
trainloader = data.DataLoader(dataset, batch_size=128, shuffle=True)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [ ]:
epoches = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Load the model to cuda device in train mode
model.to(device)
model.train()
# loop over the dataset multiple times
for epoch in range(epoches):

    running_loss = 0.0
    for i, batch in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
idx_to_calc = list(mapping.values())
idx_to_calc = np.array([idx_to_calc]).T

translator = {v: k for k, v in mapping.items()}
preds = model.embedding(torch.tensor(idx_to_calc).to(device)).cpu().detach().numpy()
plt.scatter(preds[:, 0, 0], preds[:, 0, 1], alpha=0)
for i, idx in enumerate(idx_to_calc):
    plt.text(preds[i, 0, 0], preds[i, 0, 1], translator[idx[0]])

In [ ]:
class NextLetterPrediction(torch.nn.Module):
   def __init__(self, vocab_size, embedding_size):
       super(NextLetterPrediction, self).__init__()
       self.embedding = torch.nn.Embedding(vocab_size, embedding_size)
       self.fc = torch.nn.Linear(embedding_size, vocab_size)

   def forward(self, x):
       x = F.relu(self.embedding(x))
       x = self.fc(x)
       return x